<a href="https://colab.research.google.com/github/johngeoscrub/geoscrub-google-collab-tutorials/blob/main/Interactive_Analysis_with_County_Property%C2%A0Data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install geopy
%pip install leafmap
%pip install geopandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.6/108.6 kB 10.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.6/181.6 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.1/72.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.0 MB/s eta 0:00:00


In [35]:
from geopy.geocoders import Nominatim

address = "107 Hunters Crossing Blvd, Bastrop Texas"
geolocator = Nominatim(user_agent="leafmap_example")
location = geolocator.geocode(address)

print(location.latitude, location.longitude)

30.109532199999997 -97.35715218793044


In [36]:
import leafmap.foliumap as leafmap

if location:
    latitude, longitude = location.latitude, location.longitude
else:
    print("Could not geocode the address.")

m = leafmap.Map(center=[latitude, longitude], zoom=15)
m.add_marker(location=[latitude, longitude], popup=address)

In [37]:
m

In [38]:
import requests

# URL to the Bastrop County Parcel Feature Layer endpoint - we get our data from here
feature_layer_url = "https://services3.arcgis.com/wdTkTU0MdZbNBEZy/ArcGIS/rest/services/ParcelARI/FeatureServer/0"

In [39]:
from pyproj import Proj, transform

# Specify the specific latitude and longitude
lat, lon = location.latitude, location.longitude

# Define the target CRS (EPSG:2277)
target_crs = Proj(init='epsg:2277')

# Transform the input coordinates to the target CRS
lon, lat = transform(Proj(init='epsg:4326'), target_crs, lon, lat)

# Specify your query parameters with filters for a specific location and 1-mile radius
params = {
    'where': 'land_val > 100000',
    'geometry': f'{lon},{lat}',  # Set the center point
    'geometryType': 'esriGeometryPoint',
    'spatialRel': 'esriSpatialRelIntersects',
    'distance': 1,  # Set the radius in miles
    'units': 'esriSRUnit_StatuteMile',
    'outFields': '*',  # * gets all fields
    'returnGeometry': True,  # We want Geometry to do spatial analysis
    'f': 'json',  # Response format as JSON
    'outSR': 2277  # Specify the target CRS (EPSG:2277)
}

# Make the request
response = requests.get(feature_layer_url + '/query', params=params)

In [40]:
import geopandas as gpd

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    data = response.json()

    # Access the features
    features = data.get("features", [])

    # Convert Esri JSON to GeoJSON for each feature
    for feature in features:
        geometry_esri = feature.get("geometry")
        if geometry_esri:
            geometry_geojson = {
                "type": "Polygon",
                "coordinates": geometry_esri.get("rings")
            }
            feature['geometry'] = geometry_geojson
            feature["properties"] = feature.get("attributes", {})

    # Create our GeoDataFrame
    parcel_gdf = gpd.GeoDataFrame.from_features(features, crs="epsg:2277")

    # Display the GeoDataFrame
    print(parcel_gdf.head())

else:
    print(f"Error: {response.status_code} - {response.text}")

                                            geometry  objectid prop_id_text  \
0  POLYGON ((3237697.759 10015999.516, 3237485.35...     32369       115192   
1  POLYGON ((3237831.802 10016291.820, 3237555.59...     32370        30102   
2  POLYGON ((3237838.229 10016292.005, 3237831.80...     42839      8706380   
3  POLYGON ((3237667.225 10015849.543, 3237665.33...     36647        95379   
4  POLYGON ((3237686.907 10015963.084, 3237678.04...     32444       114958   

  created_date last_edited_date  prop_id  owner_tax_yr  \
0         None             None   115192          2024   
1         None             None    30102          2024   
2         None             None  8706380          2024   
3         None             None    95379          2024   
4         None             None   114958          2024   

                         file_as_name  legal_acreage   hood_cd  ...  \
0                          KAMRAJ LLC          1.501  NBHD0313  ...   
1               LIRTEX PROPERTIES 

In [41]:
m = leafmap.Map(center=[latitude, longitude], zoom=15)

In [42]:
# Reporject geodataframe to 4326
gdf_4326 = parcel_gdf.to_crs(epsg='4326')

# Add GeoPandas DataFrame to the map
m.add_gdf(gdf_4326, layer_name='Polygons')

In [43]:
m
